In [4]:
# Import packages
import os
import geopandas as gpd
import numpy as np
import rasterio
import rasterio.mask

In [5]:
# Import this file to derive the polygon geometry for Telangana
bound_Telangana = gpd.read_file('C:/Users/mieke/Documents/Msc Thesis/GitHub repositories/dicra/analytics/geospatial_internship/datasets/telangana_shapefile.geojson')
bound_Telangana.head()

,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry
0,IND,India,IND.32_1,Telangana,None,None,State,State,None,IN.TG,"MULTIPOLYGON (((79.21147 16.36281, 79.22324 16..."


In [6]:
# Create a Polygon geometry of Telangana
bound_Telangana = bound_Telangana.explode()
bound_Telangana.head()

C:\Users\mieke\AppData\Local\Temp\ipykernel_22400\3231266139.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  bound_Telangana = bound_Telangana.explode()


,,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry
0,0,IND,India,IND.32_1,Telangana,None,None,State,State,None,IN.TG,"POLYGON ((79.21147 16.36281, 79.22324 16.35036..."


In [12]:
# For each of the LAI images, we crop the Geotiff file to Leaf Area Index in Telangana
folder_names = os.listdir('C:/Users/mieke/Documents/Msc Thesis/Notebooks Python/LAI/Input/LAI images')
for i in folder_names:
    file_names = os.listdir('C:/Users/mieke/Documents/Msc Thesis/Notebooks Python/LAI/Input/LAI images/' + i)
    for j in file_names:
        LAI_data = "C:/Users/mieke/Documents/Msc Thesis/Notebooks Python/LAI/Input/" + i + "/" + j
        with rasterio.open(LAI_data) as src:
            out_image, out_transform = rasterio.mask.mask(src, bound_Telangana.geometry, crop=True)
            out_meta = src.meta

        out_meta.update({"driver": "GTiff",
                        "height": out_image.shape[1],
                        "width": out_image.shape[2],
                        "transform": out_transform})

        # photometric = 'YCbCr'
        # photometric = 'RGB', compress = 'JPEG'
        # Save the Geotiff file to Leaf Area Index in Telangana
        with rasterio.open("C:/Users/mieke/Documents/Msc Thesis/Notebooks Python/LAI/Output/Telangana_" + j, "w", **out_meta) as dest:
            dest.write(out_image)